In [2]:
import pyaudio
import wave
import sys

In [12]:
CHUNK = 1024
audio_path = "F:/Capstone Project/IEMOCAP-Emotion-Detection/data/CMU_MOSI/Raw/Audio/WAV_11025/"
file_name = "PZ-lDQFboO8.wav"

wf = wave.open(audio_path + file_name, 'rb')


In [17]:
# instantiate PyAudio (1)
p = pyaudio.PyAudio()

# open stream (2)
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

# read data
data = wf.readframes(CHUNK)

In [18]:
# play stream (3)
while len(data) > 0:
    stream.write(data)
    data = wf.readframes(CHUNK)


KeyboardInterrupt: 

In [15]:
# stop stream (4)
stream.stop_stream()
stream.close()

# close PyAudio (5)
p.terminate()